In [49]:
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import time
import re

# decorator 정의
def checktime(label=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            func_name = label or func.__name__
            print(f"[{func_name}] 실행 시간: {end - start:.4f}초")
            return result
        return wrapper
    return decorator

def log_call(func):
    def wrapper(*args, **kwargs):
        print(f"[LOG] {func.__name__} 함수가 호출되었습니다.")
        return func(*args, **kwargs)
    return wrapper

# 예상 결과 그래프 출력 클래스
class ResultGrapher:
    default_color = 'skyblue'

    @classmethod
    def set_color(cls, color):
        cls.default_color = color

    @staticmethod
    def plot_bar(series: pd.Series, master_window: tk.Tk, text_summary: str = None):
        plt.rcParams['font.family'] = 'Malgun Gothic'
        plt.rcParams['axes.unicode_minus'] = False

        popup = tk.Toplevel(master_window)
        popup.title("Predicted Outcome Chart")
        popup.geometry("600x580")

        fig, ax = plt.subplots(figsize=(6, 4))
        series.plot(kind='bar', ax=ax, color=ResultGrapher.default_color)
        ax.set_ylabel("Probability (%)")
        ax.set_title("Predicted Outcome Chart")
        ax.set_ylim(0, 100)
        plt.xticks(rotation=45)
        fig.tight_layout()

        canvas = FigureCanvasTkAgg(fig, master=popup)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=False, pady=(10, 0))

        if text_summary:
            label = tk.Label(popup, text=text_summary.strip(), justify="left", font=("Arial", 9))
            label.pack(padx=10, pady=(5, 5), anchor="w")

        tk.Button(popup, text="닫기", command=popup.destroy).pack(pady=5)

# 입력한 선수에 대한 클래스
class Player:
    def __init__(self, name, team, stats):
        self.name = name
        self.team = team
        self.stats = stats

    @property
    def info(self):
        text = f"이름: {self.name}\n구단: {self.team}\n\n"
        for k, v in self.stats.items():
            text += f"{k}: {v}\n"
        return text

# 입력받을 투수와 타자에 대한 클래스 형성
class Pitcher(Player):
    pass

class Batter(Player):
    @property
    def home_run_rate(self):
        pa = self.stats.get("PA", 0)
        hr = self.stats.get("HR", 0)
        return (hr / pa * 100) if pa else 0

# 트레이드 시스템을 위한 클래스
class TeamChanger:
    def __init__(self, filepath):
        self.filepath = filepath
        self.df = pd.read_excel(filepath, engine="openpyxl")

    def get_teams(self):
        return sorted(self.df['Team'].dropna().unique())

    def get_players_by_team(self, team):
        return self.df[self.df['Team'] == team]['Name'].tolist()

    def change_team(self, name, current_team, new_team):
        idx = self.df[(self.df['Name'] == name) & (self.df['Team'] == current_team)].index
        if not idx.empty:
            self.df.at[idx[0], 'Team'] = new_team
            self.df.to_excel(self.filepath, index=False)
            return True
        return False

#트레이드 시스템 실행 클래스 구현
class TradeApp(tk.Toplevel):
    def __init__(self, master):
        super().__init__(master)
        self.title("선수 트레이드 메뉴")
        self.geometry("400x500")
        self.main_menu()

    def main_menu(self):
        self.clear_window()
        tk.Label(self, text="트레이드 메뉴", font=("Arial", 16, "bold")).pack(pady=20)
        tk.Button(self, text="1. 투수 팀 변경", command=lambda: self.trade_window("pitcher.xlsx")).pack(pady=10)
        tk.Button(self, text="2. 타자 팀 변경", command=lambda: self.trade_window("fielder.xlsx")).pack(pady=10)
        tk.Button(self, text="3. 닫기", command=self.destroy).pack(pady=20)

    def trade_window(self, filepath):
        self.clear_window()
        changer = TeamChanger(filepath)
        teams = changer.get_teams()

        tk.Label(self, text="기존 팀 선택").pack()
        old_team = ttk.Combobox(self, values=teams, state="readonly"); old_team.pack(pady=5)

        tk.Label(self, text="선수 선택").pack()
        player_combo = ttk.Combobox(self, state="readonly"); player_combo.pack(pady=5)

        def update_players(*args):
            player_combo["values"] = changer.get_players_by_team(old_team.get())

        old_team.bind("<<ComboboxSelected>>", update_players)

        tk.Label(self, text="새 팀 선택").pack()
        new_team = ttk.Combobox(self, values=teams, state="readonly"); new_team.pack(pady=5)

        def do_trade():
            player, old, new = player_combo.get(), old_team.get(), new_team.get()
            if not (player and old and new):
                messagebox.showwarning("입력 오류", "모든 정보를 입력하세요"); return
            if old == new:
                messagebox.showwarning("입력 오류", "같은 팀으로 변경 불가"); return
            if changer.change_team(player, old, new):
                messagebox.showinfo("성공", f"{player}의 팀이 {old} → {new}로 변경됨")
                self.main_menu()
            else:
                messagebox.showerror("실패", "팀 변경 실패")

        tk.Button(self, text="확정", command=do_trade).pack(pady=15)
        tk.Button(self, text="메인으로", command=self.main_menu).pack()

    def clear_window(self):
        for widget in self.winfo_children():
            widget.destroy()

# 앱을 실행하기 위해 틀을 위한 클래스 구현
class BaseballApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Final Project")
        self.geometry("600x450")

    def clear_window(self):
        for widget in self.winfo_children():
            widget.destroy()

    def exit_program(self):
        self.quit()
        self.destroy()


#이닝 시작 메뉴에 대한 클래스 구현        
class InningManager(BaseballApp):
    def __init__(self):
        super().__init__()
        self.pitcher_info = None
        self.batter_info = None
        self.create_main_menu()

    def create_main_menu(self):
        self.clear_window()
        tk.Label(self, text="Baseball Simulator", font=("Arial", 20, "bold", "italic")).pack(pady=50)
        tk.Button(self, text="1. 이닝 시작", command=self.create_inning_menu).pack(pady=10)
        tk.Button(self, text="2. 선수 트레이드 메뉴", command=lambda: TradeApp(self)).pack(pady=10)
        tk.Button(self, text="3. 프로그램 종료", command=self.exit_program).pack(pady=80)

    def create_inning_menu(self):
        self.clear_window()
        tk.Label(self, text="이닝 진행 중", font=("Arial", 14, "bold")).pack(pady=50)

        if self.pitcher_info:
            frame1 = tk.Frame(self); frame1.pack(pady=5)
            tk.Button(frame1, text="1. 투수 정보 출력", command=lambda: messagebox.showinfo("투수 정보", self.pitcher_info.info)).pack(side="left", padx=10)
            tk.Button(frame1, text="초기화", command=self.reset_pitcher_info).pack(side="left", padx=5)
        else:
            tk.Button(self, text="1. 투수 정보 입력", command=self.enter_pitcher_info).pack(pady=10)

        if self.batter_info:
            frame2 = tk.Frame(self); frame2.pack(pady=5)
            tk.Button(frame2, text="2. 타자 정보 출력", command=lambda: messagebox.showinfo("타자 정보", self.batter_info.info)).pack(side="left", padx=10)
            tk.Button(frame2, text="초기화", command=self.reset_batter_info).pack(side="left", padx=5)
        else:
            tk.Button(self, text="2. 타자 정보 입력", command=self.enter_batter_info).pack(pady=10)

        if self.pitcher_info and self.batter_info:
            tk.Button(self, text="3. 예상 결과 분석", command=self.analyze_result).pack(pady=10)
            offset = 1
        else:
            offset = 0

        tk.Button(self, text=f"{3 + offset}. 이전 메뉴로 돌아가기", command=self.create_main_menu).pack(pady=10)
        tk.Button(self, text=f"{4 + offset}. 프로그램 종료", command=self.exit_program).pack(pady=45)

    def enter_pitcher_info(self):
        self.open_input_popup("투수 정보 입력", self.save_pitcher_info)

    def enter_batter_info(self):
        self.open_input_popup("타자 정보 입력", self.save_batter_info)

    def open_input_popup(self, title, save_callback):
        popup = tk.Toplevel(self); popup.title(title); popup.geometry("300x180")
        is_pitcher = "투수" in title
        try:
            df = pd.read_excel("pitcher.xlsx" if is_pitcher else "fielder.xlsx", engine="openpyxl")
            name_list = sorted(df['Name'].dropna().unique().tolist())
            team_list = sorted(df['Team'].dropna().unique().tolist())
        except Exception as e:
            messagebox.showerror("파일 오류", str(e)); popup.destroy(); return

        tk.Label(popup, text="이름 선택:").pack()
        name_combo = ttk.Combobox(popup, values=name_list, state="readonly")
        name_combo.pack()

        tk.Label(popup, text="소속 구단 선택:").pack()
        team_combo = ttk.Combobox(popup, values=team_list, state="readonly")
        team_combo.pack()

        def save():
            name, team = name_combo.get().strip(), team_combo.get().strip()
            if name and team:
                save_callback(name, team); popup.destroy(); self.create_inning_menu()
            else:
                messagebox.showwarning("입력 오류", "모든 정보를 선택해주세요.")

        tk.Button(popup, text="저장", command=save).pack(pady=10)

    @checktime("투수 정보 저장")
    def save_pitcher_info(self, name, team):
        df = pd.read_excel("pitcher.xlsx", engine="openpyxl")
        row = df[(df['Name'] == name) & (df['Team'] == team)]
        if row.empty:
            messagebox.showerror("오류", f"{name} ({team}) 정보가 없습니다."); return
        stats = {col: row.iloc[0][col] for col in ['ERA', 'WHIP', 'IP', 'SO', 'BB', 'K/BB', 'HR']}
        self.pitcher_info = Pitcher(name, team, stats)

    @checktime("타자 정보 저장")
    def save_batter_info(self, name, team):
        df = pd.read_excel("fielder.xlsx", engine="openpyxl")
        row = df[(df['Name'] == name) & (df['Team'] == team)]
        if row.empty:
            messagebox.showerror("오류", f"{name} ({team}) 정보가 없습니다."); return
        stats = {col: row.iloc[0][col] for col in ['AVG', 'OBP', 'SLG', 'OPS', 'PA', 'H', 'HR', 'RBI', 'BB', 'SO']}
        self.batter_info = Batter(name, team, stats)

    @log_call
    def reset_pitcher_info(self):
        self.pitcher_info = None
        messagebox.showinfo("초기화", "투수 정보가 삭제됐습니다.")
        self.create_inning_menu()

    @log_call
    def reset_batter_info(self):
        self.batter_info = None
        messagebox.showinfo("초기화", "타자 정보가 삭제됐습니다.")
        self.create_inning_menu()

    @checktime("예상 결과 분석")
    @log_call
    def analyze_result(self):
        p, b = self.pitcher_info.stats, self.batter_info.stats
        pa = b['PA']; h = b['H']; bb = b['BB']; so = b['SO']; hr_b = b['HR']; ip = p['IP']; hr_p = p['HR']

        hit = h / pa * 100 if pa else 0
        walk = bb / pa * 100 if pa else 0
        hr_b_rate = hr_b / pa * 100 if pa else 0
        strike = so / pa * 100 if pa else 0
        hr_p_rate = hr_p / ip * 100 if ip else 0
        out = max(0, 100 - (hit + walk))
        hr_comb = (hr_b_rate + hr_p_rate) / 2

        series = pd.Series({"Hit": hit, "Walk": walk, "Strikeout": strike, "Home Run": hr_b_rate, "HR/IP": hr_p_rate, "Combined HR": hr_comb, "Out": out})
        summary = f"""
[예측치]

1 안타 확률: {hit:.1f}%
2 볼넷 확률: {walk:.1f}%
3 삼진 확률: {strike:.1f}%
4 홈런 확률: {hr_b_rate:.1f}%
5 투수 HR/IP: {hr_p_rate:.1f}%

=> 홈런 예측치: {hr_comb:.1f}%
=> 아웃 확률: {out:.1f}%
"""
        ResultGrapher.plot_bar(series, self, summary)

if __name__ == "__main__":
    app = InningManager()
    app.mainloop()
